# Extract data from caNano exported json file
Rui He, 5/19/2022

for my dear Weina Miao Miao

In [68]:
import json
import os
import csv

data_path=os.path.abspath("caNanoLab_sample_data_1652929321540.json")
output_path=os.path.abspath("Function_extract.csv")
data={}

In [69]:
with open(data_path, 'r',errors='ignore') as file:
    data_temp = file.read()
    data = json.loads(data_temp)


In [70]:
def check_key_exist(dictionary, key):     
    if dictionary.get(key) is not None:
        return True
    else:
        return False

In [71]:
def Check_and_extract(dictionary,key,palceholder):
    if dictionary.get(key) is not None:
        targetvar=dictionary[key]
    else:
        targetvar=palceholder
    return targetvar
        

In [72]:
headers=['samplename','nameid','function entity','property name', \
        'property type','function type','function description','full description']
output_table=[]

In [83]:
counter=1
for item in data:
    

    sampleId=item['sample']['GeneralInfo']['sampleId']
        
    sampleName=item['sample']['GeneralInfo']['sampleName']

    funcEntity=item['sample']['Composition']['functionalizingentity']
    
    # if sampleId==11337728:
    #     print(counter)
    #     break
    if len(funcEntity)!=0:
        for item in funcEntity:
            for subitem in funcEntity[item]:
                prop_name=subitem['Name']                
                full_des=subitem['Desc']
                
                if check_key_exist(subitem,'Properties'):
                    if len(subitem['Properties'])==0:
                        prop_type='NaN'
                    else: 
                        prop_type=Check_and_extract(subitem['Properties'], 'type','NaN')
                else:
                    prop_type='NaN'
                
                
                if check_key_exist(subitem,'Functions'):
                    if len(subitem['Functions'])==0:
                        func_type='NaN'
                        func_des='NaN'
                    else:
                        for fuc_item in subitem['Functions']:
                            func_type=fuc_item['Type']                        
                            func_des=Check_and_extract(fuc_item, 'FunctionDescription','NaN')   
                            
                            to_write=[sampleName,sampleId,item,prop_name,prop_type,func_type,func_des,full_des]
                            if len(output_table)>0:
                                if output_table[-1]!=to_write:
                                    output_table.append(to_write) 
                            else:
                                output_table.append(to_write)
                            
                else:
                    func_type='NaN'
                    func_des='NaN'
                    to_write=[sampleName,sampleId,item,prop_name,prop_type,func_type,func_des,full_des]
                    if len(output_table)>0:
                        if output_table[-1]!=to_write:
                            output_table.append(to_write)
                    else:
                        output_table.append(to_write)
                
    else:
        to_write=[sampleName,sampleId,'NaN','NaN','NaN','NaN','NaN','NaN']
        if len(output_table)>0:
            if output_table[-1]!=to_write:
                output_table.append(to_write)
        else:
            output_table.append(to_write)
    counter+=1

                


In [84]:
# output_table[10]

In [85]:
for item in output_table:
    tracer='UNC-HKangPR2005-05'
    if tracer in item:
        print(item)

['UNC-HKangPR2005-05', 11337730, 'small molecule', 'BODIPY', 'NaN', 'imaging function', 'Imaging dendrimer conjugates incubated in NIH 3T3 MDR cells', 'Fluorophore\r\n4.219 micromol\r\nPubChem CID 25058173']
['UNC-HKangPR2005-05', 11337730, 'biopolymer', 'Tat', 'peptide', 'other', 'Transfection of dendrimer conjugates into NIH 3T3 MDR cells', 'Cell penetrating peptide and trans-activating transcriptional activator\r\nfrom Human Immunodeficiency Virus 1\r\nTat peptide with a cysteine residue at C terminus (NH2-RKKRRQRRRPPQGGC-COOH)\r\n13.30 micromol\r\nAverage number of peptides 15.9 ']
['UNC-HKangPR2005-05', 11337730, 'biopolymer', 'Antisense Cy5-5995', 'DNA', 'targeting function', 'Targeting the AUG start codon of the MDR1 gene', "Antisense 20 base pair phosphorothioate oligonucleotide with a sequence targeting AUG start codon of MDR1 gene and attached fluorophore\r\n5'-(Cy5)CCA TCC CGA CCT CGC GCT CC-3'\r\nG5.NH2 + BODIPY 20 microg/mL"]
['UNC-HKangPR2005-05', 11337730, 'biopolymer', 

In [86]:
with open(output_path, 'w',newline="") as output:
    writer = csv.DictWriter(output, fieldnames = headers)
    writer.writeheader()
    writer = csv.writer(output)
    writer.writerows(output_table)